# Welcome to MoSeq2-Notebook

### Run all of the MoSeq tools in a containerized notebook.

# Setup

To begin, run the following jupyter notebook to ensure MoSeq2 is installed and running smoothely on your machine.
[SETUP NOTEBOOK](http://localhost:8889/notebooks/MoSeq2_Step_0.ipynb)

Next, copy this notebook file to your recorded session directory. You will create a new copy of this notebook for each analysis session.

You can use the cells below to ensure your sessions are found in this notebook, and the correct python is being used in your designated conda env (where your moseq2 tools are installed).

In [ ]:
import os
os.listdir(os.getcwd())

Ensure you are running the python version located in your corresponding conda environment.

In [ ]:
%%bash
which python

## Generate Configuration Files

If the files and env are correct, run the cell below to generate a configuration file that will aid in modifying advanced analysis/extraction parameters throughout the MoSeq2 pipeline.

### Single Session Configuration File

In [ ]:
from moseq2_extract.gui import *

output_filepath = './config.yaml'

generate_config_command(output_filepath)

### Multiple Session (Batch) Configuration File

In [ ]:
import os
from moseq2_extract.gui import *
#create a config file for each individual session
sessions = [x for x in os.listdir(os.getcwd()) if (os.path.isdir(x) and x[0] != '.')]
print(sessions)

curr_dir = os.getcwd()

for sess in sessions:
    generate_config_command(sess+'/config.yaml')

In [ ]:
import os
from moseq2_batch.gui import *

input_dir = '/Users/AymanZAY/Desktop/moseq2/moseq2-batch/scan_file_samples'
output_dir = os.getcwd()
name = 'gridsearch-config.yaml'
scan_type = 'gauss'

gridsearch_config_command(input_dir, output_dir, name, scan_type)

## Download a Flip file
In order to ensure your extraction is smooth and invariant to the mouse's orientation, we recommend using a flip-classifier to aid keeping the mouse oriented throughout the extraction.

The flip file indices are as follows:
* [0] - Large mice with fibers.
* [1] - Adult male c57s.
* [2] - Mice with Inscopix cables.

Enter your desired index in the variable assignment below and run the cell.

In [ ]:
from moseq2_extract.gui import *

output_dir = './'
config_file = './config.yaml'
selected_index = 1
download_flip_command(output_dir, config_file, selected_index)

Once that is done, (__for single sessions__) the flip file will be automatically used in your following extractions.

For mulitple sessions (for now) you will have to manually input your desired flip file.

# Raw Data Extraction Step

To extract your current session's depth file, input it's path below and run the cell.

__Note__: In order to configure your extraction parameters, you must edit your config.yaml file prior to executing the extract command.

## Pre-Extraction Data Quality Testing

Before performing a full extraction on your recordings, follow the following steps to ensure your Regions of Interest (ROIs) are properly found. This will bring more clarity as to what to expect after a complete extraction of your data. 

### ROI Test

In [ ]:
from moseq2_extract.gui import *

input_file = 'first_test/depth.dat'
output_dir = 'sample_proc/'
config_file = 'config.yaml'

find_roi_command(input_file, output_dir, config_file)

In [ ]:
#Display extracted ROI
from IPython.display import display, Image
display(Image(output_dir+'bground.tiff'))

### Sample Test Extraction 
Run the following cell to test your raw data extraction parameters before extracting all of your data to ensure the best data quality going into the modeling step.

In [ ]:
from moseq2_extract.gui import *

input_file = 'first_test/depth.dat'
output_dir = 'test_proc/'
config_file = 'config.yaml'
nframes = 100

sample_extract_command(input_file, output_dir, config_file, nframes)

from IPython.display import Video, Image
Video('first_test/'+output_dir+'results_00.mp4')

## Single Session Extraction
If you only have one session you would like to extract, run the following cell. Otherwise, run the Batch Extraction cell.

In [ ]:
from moseq2_extract.gui import *

input_file = 'depth.dat'
output_dir = 'proc/'
config_file = 'config.yaml'

extract_command(input_file, output_dir, config_file)

from IPython.display import Video, Image
Video(output_dir+'results_00.mp4')

## Multi-Session (Batch) Extraction
Extract multiple recording sessions (directories).

In [ ]:
from moseq2_batch.gui import *
from pathlib import Path
import os

input_dir = './'
config_file = Path('./gridsearch-config.yaml')
filename = 'depth.dat'
cluster_type = 'local'

## advanced settings
temp_storage = Path('tmp/')
ncpus = 4
memory = "10GB"
wall_time = '3:00:00'
partition = 'short'
bg_roi_index = [0]
prefix = ''
skip_checks = False

commands = extract_batch_command(input_dir, config_file, filename, cluster_type, temp_storage,
                  ncpus, memory, wall_time, partition, bg_roi_index, prefix, skip_checks)

with open('batch_extract.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    for cmd in commands:
        cmd = cmd.strip(';')
        f.write('%s\n' % cmd)

os.system('chmod a+x batch_extract.sh')

Run the following bash command to execute your batch extraction:

In [ ]:
%%bash
./batch_extract.sh

Once that is done, aggregate all of your extraction results to consolidate all of your metadata and timestamp data in one folder.

In [ ]:
from moseq2_batch.gui import *

input_dir = './'
ex_format = '{start_time}_{session_name}_{subject_name}'
output_dir = 'aggregate_results/'
mouse_threshold = 0

aggregate_extract_results_command(input_dir, ex_format, output_dir, mouse_threshold)

# Principal Component Analysis (PCA) Step

Once all your data is extracted and saved in your desired proc/ directory, you are now able to perform the PCA step.

## Training

To train your PCA on your extracted data results, run the following command. It will recursively search within your current directory structure for your extracted results_xx.h5 files.

In [ ]:
from moseq2_pca.gui import *

input_dir = './'
config_file = 'config.yaml'
output_file = 'pca'
output_dir = '_pca/'

train_pca_command(input_dir, config_file, output_dir, output_file)

from IPython.display import display, Image
images = [output_dir+'pca_components.png',output_dir+'pca_scree.png']
for im in images:
    display(Image(im))

## Computing Principal Component Scores
Once your PCA model has been trained, you can now apply your model using your extracted data amd computed principal components. To compute your PC Scores, run the following command:

In [ ]:
from moseq2_pca.gui import *

input_dir = './'
config_file = 'config.yaml'
output_dir = '_pca/'
output_file = 'pca_scores'

apply_pca_command(input_dir, config_file, output_dir, output_file)

## Computing Model-free Syllable Changepoints
To measure block duration distances between detected syllables using your PCA model or computed scores, you can run the following command:

In [ ]:
from moseq2_pca.gui import *

input_dir = './'
config_file = 'config.yaml'
output_dir = '_pca/'
output_file = 'changepoints'

compute_changepoints_command(input_dir, config_file, output_dir, output_file)

from IPython.display import display, Image
display(Image('_pca/changepoints_dist.png'))

# Train ARHMM (Compute Locally)

(__scroll down for batch model training__)

Once you have computed your PCA Scores, you can now use this data as your input to train your Auto-Regressive Heuristic Markov Model (ARHMM).
If you have multiple groups (for example, a control and experimental group) that you would like to model separately using the same model, use the ```--separate-trans``` flag in the command below.

## Single Group Training

In [ ]:
from moseq2_model.gui import *
import os

cwd = os.getcwd()
scores_file = cwd+'/_pca/pca_scores.h5'
dest_file = cwd+'/model.p'
config_file = './config.yaml'
index_file = ""
hold_out = False
hold_out_seed = -1
nfolds = 5
num_iter = 1
max_states = 50
npcs = 10
kappa = 100000 #nframes
gamma = 1e3
alpha = 5.7
separate_trans = False
robust = False
checkpoint_freq = -1


learn_model_command(scores_file, dest_file, config_file, index_file, hold_out, nfolds,
                    num_iter, max_states, npcs, kappa, gamma, alpha, 
                    separate_trans, robust, checkpoint_freq)

## Multiple Group Training
In order to model multiple groups separately in your model, you must generate an index file to point to all your relevant paths, as well as indicate use the separate transition graphs flag.

Begin by generating your index file:

### Generate your index file

In [ ]:
from moseq2_viz.gui import *

input_dir = './'
pca_file = '_pca/pca_scores.h5'
output_file = 'moseq2-index.yaml'
filter_tup = ()
all_uuids = False

generate_index_command(input_dir, pca_file, output_file, filter_tup, all_uuids)

In the following cell you can view which groups your subjects are associated with.

### Add your subjects to groups in your index file

In [ ]:
from moseq2_viz.gui import *

index_file = 'moseq2-index.yaml'
key = ''
value = ''
group = ''
exact = False
lowercase = False
negative = False

add_group_command(index_file, key, value, group, exact, lowercase, negative)


# Implement view groups function


### Train Multiple Groups
Now you can train your model on multiple groups using your augmented index file.

In [ ]:
from moseq2_model.gui import *
import os

cwd = os.getcwd()
scores_file = cwd+'/_pca/pca_scores.h5'
dest_file = cwd+'/model.p'
config_file = './config.yaml'
index_file = ""
hold_out = False
hold_out_seed = -1
nfolds = 5
num_iter = 100
max_states = 50
npcs = 10
kappa = 100000 #nframes
gamma = 1e3
alpha = 5.7
separate_trans = True
robust = False
checkpoint_freq = -1


learn_model_command(scores_file, dest_file, config_file, index_file, hold_out, nfolds,
                    num_iter, max_states, npcs, kappa, gamma, alpha, 
                    separate_trans, robust, checkpoint_freq)

# Train ARHMM (Batch Mode/Slurm-Enabled)

Before running the batch modeling commands, ensure that the paths in your __`gridsearch-config.yaml`__ file are correct.

## Single Group - Multi-Model Training
With batch mode, you can train multiple ARHMMs with varying parameter settings to perform comparative analysis on your model fit.

In [ ]:
from moseq2_batch.gui import *

scan_file = 'gridsearch-config.yaml'
cluster_type = 'local'
restarts = 1
var_name = 'scores'
image = 'model-image'
job_name = 'batchjob'
output_dir = 'batchjobs/'
ext = '.p'
mount_point = ''
bucket = 'bucket'
restart_policy = 'OnFailure'
ncpus = 7
memory = '14GB'
input_file = 'use_data.mat'
check_cluster = ''
log_path = 'logs/'
kind = 'Job'
preflight = False
copy_log = True
skip_checks = False
start_num = 0
partition = 'debug'
wall_time = '11:00:00'


commands = learn_model_parameter_scan_command(scan_file, cluster_type, restarts, var_name, image, job_name,
                               output_dir, ext, mount_point, bucket, restart_policy,
                               ncpus, memory, input_file, check_cluster, log_path,
                               kind, preflight, copy_log, skip_checks, start_num, partition, wall_time)

with open('train_batches.sh', 'w') as f:
    f.write('#!/bin/bash\n')
    for cmd in commands:
        cmd = cmd.strip(';')
        cmd = cmd[:-1]
        f.write('%s\n' % cmd)

os.system('chmod a+x train_batches.sh')

### Execute your batch model training

In [ ]:
%%bash
./train_batches.sh

In [ ]:
from moseq2_batch.gui import *

input_dir = './_pca/modeling_results/batchjob_2019-09-29_22-53-35'
job_manifest = None
dest_file = 'export_results.h5'

aggregate_modeling_results_command(input_dir, job_manifest, dest_file)

# Visualize Results

Now that you have a trained ARHMM, you can use the moseq2-viz module to produce crowd videos and a number of statistical analysis plots.

## Setup
Ensure that you have a `moseq2-index.yaml` file generated using this command:

In [ ]:
from moseq2_viz.gui import *

input_dir = './'
pca_file = '_pca/pca_scores.h5'
output_file = 'moseq2-index.yaml'
filter_tup = ()
all_uuids = False

generate_index_command(input_dir, pca_file, output_file, filter_tup, all_uuids)

## Make Crowd Videos
This tool allows you to create videos containing many overlayed clips of the mouse performing the same specified syllable at the moment a red dot appears on their body. The videos are sorted by most frequently expressed syllable to least.
To create the crowd videos, run the following command:

In [ ]:
from moseq2_viz.gui import *

index_filepath = 'moseq2-index.yaml'
model_path = 'model.p'
config_file = './config.yaml'
output_dir = './crowd_movies/'
max_syllables, max_examples = 10, 10

make_crowd_movies_command(index_filepath, model_path, config_file, output_dir, max_syllables, max_examples)

## Compute Usage Plots
Use this command to compute the model-detected syllables usages sorted in descending order of usage.

In [ ]:
from moseq2_viz.gui import *

index_filepath = 'moseq2-index.yaml'
model_fit = 'model.p'
sort = True
count = 'usage'
max_syllable = 40
group = ''
output_file = 'usages'

plot_usages_command(index_filepath, model_fit, sort, count, max_syllable, group, output_file)
from IPython.display import display, Image
display(Image('usages.png'))

## Compute Scalar Summary and Tracking Plots
Use the following command to compute some scalar summary information about your modeled groups, such as average velocity, height, etc.
This command will also generate a tracking summary plot; depicting the path traveled by the mouse in your recordings.

In [ ]:
from moseq2_viz.gui import *

index_filepath = 'moseq2-index.yaml'
output_file = 'scalars'

plot_scalar_summary_command(index_filepath, output_file)
from IPython.display import display, Image
display(Image('scalars_summary.png'))
display(Image('scalars_position.png'))

## Compute Syllable Transition Graph
Use the following command to generate a syllable transition graph. The graph will be comprised of nodes labelled by syllable, and edges depicting a probable transition, with edge thickness depicting the weight of the transition edge.

For multiple groups, there will be a transition graph for each group, as well as a unified graph with different colors to identify the groups.

In [ ]:
from moseq2_viz.gui import *

index_filepath = 'moseq2-index.yaml'
model_fit = 'model.p'
config_file = './config.yaml'
max_syllable = 40
group = ''
output_file = 'transition'


plot_transition_graph_command(index_filepath, model_fit, config_file, max_syllable, group, output_file)

from IPython.display import display, Image
display(Image('transition.png'))